In [1]:
demand = [990,1980,3961,2971,1980]   
d=0  # d% shortage allowance
Y_b = [1.3086,1.3671,1.4183,1.4538,1.5122]           # Fabric yield (consumption rate) rate per garment of size 𝛽
f= 1.95 # Fabric cost 


U = 1
l_max= 20  #yrds
e=0


if len(demand)!=len(Y_b):
    raise ValueError('number of sizes and number of fabric consumption does not match')

#Input variables (setup cost)

c_s=500  #setup cost
c_e=20. #excess production cost

P_min, P_max= 4,40

In [2]:
import numpy as np
import math
from copy import deepcopy
import random
import time
rng = np.random.default_rng()
import pandas as pd

In [3]:
def Shortage_allowance(Q,d=0.01):
    temp=np.dot((1-d),Q)
    return [round(i) for i in temp]
Q_b= Shortage_allowance(demand,d)

# Q_b

In [4]:
def Update_Res(R,GG,PP):
    for s in range(len(GG)): #Updating Residual within the while loop
        R=R-np.dot(GG[s],PP[s])
    return R

In [5]:
def Length(g_i_j):
    l_i = e+ np.dot(g_i_j,Y_b)/U
    return l_i


In [6]:
from Heuristics import H1,H3,H5,H3_MB2016,H1_T2020


## Crossover from (M'Hallah and Bouziri, 2016, p. 338)

In [7]:
def Crossover(Parent_1,Parent_2,crossover_rate=1):
    '''
    Parent_1,Parent_2 are dictionary containing G_a_b & P_a as values    
    '''
    
    if rng.random() > crossover_rate:
        return Parent_1

    
    G1 = Parent_1['G']
    P1 = Parent_1['P'] 
    G2 = Parent_2['G']
    P2 = Parent_2['P']
    #print('P1:',P1)

    m_1=len(G1)
    Beta=len(G1[0])
    
    
    cp=rng.integers(0,m_1,size=2)
    cp.sort()    #cp is the cutpoint for crossover
    #print('cp:', cp)
    C_G= G1[cp[0]:cp[1]+1]
    C_P= P1[cp[0]:cp[1]+1]
    #print(C_G,C_P)

    '''a few section is taken from Parent 1'''
        

    
    r_b= Update_Res(Q_b, C_G, C_P)
    #print('r_b:',r_b)
    
    '''r_b is the residual demand after few section taken from Parent1'''
    
    i=0
    ''' i is the index number of sections(G) from Parent_2'''
    
    while max(r_b)>0:
        #print(f'\nresidual demand:{r_b}')
        
        g_i_j= list (np.zeros(Beta,dtype=int))
        
        if i<=(len(P2)-1) and max(r_b)>P_min:     
            
            K=[]
            for j in range(Beta):
                if G2[i][j]>0:
                    K.append(j)
            '''K in the list of index where g_i_j of Parent2 is greater than 0'''
            
            #print(f'G2: {G2} and K:{K}')
            try:
                GCD= np.gcd.reduce([r_b[k] for k in K if r_b[k]>0])
            except:  #if the list is null
                GCD=1 
            '''GCD is LCD (r_s : r_s > 0, O_i_s > 0, s ∈ S)  
            from (M'Hallah and Bouziri, 2016, p. 338)'''
            
           
            p_i = max(P_min,min(P2[i],GCD)) ###
            
            #print(f'-{p_i}-')
            for j in K:
                '''K in the list of index where g_i_j of Parent2 is greater than 0'''
                if r_b[j]>0:
                    g_i_j[j]=min(math.floor(r_b[j]/p_i),math.floor((l_max-Length(g_i_j))/(Y_b[j]/U)))
                '''else 0'''
            
            r_b= r_b-np.dot(g_i_j,p_i)


                
                
            if max(g_i_j)>0:   #This is to avoid all zero section from adding
                C_G.append(g_i_j)
                C_P.append(p_i)
            i+=1    # goto the next section of Parent2
            
        else:
            p_i=min(P_max,max(P_min,np.gcd.reduce([k for k in r_b if k>0])))
            #print(f'+{p_i}+')
            for j in range(Beta):
                if r_b[j]>0:
                    g_i_j[j]=min(math.ceil(r_b[j]/p_i), math.floor((l_max-Length(g_i_j))/(Y_b[j]/U)))
                
                '''else 0 is already there '''


                
            r_b= r_b-np.dot(g_i_j,p_i)
            
            if max(g_i_j)>0:
                C_G.append(g_i_j)
                C_P.append(p_i)
            i+=1
            #break
            
    return {'G':C_G,'P':C_P}
    
# C=Crossover(Sol_1,Sol_2)
# print(C)

## Mutation Operator from M&B 2016

In [8]:
def mutate(ii,jj,G,P):    

    G[ii][jj]=0
    r = Update_Res(Q_b,G,P)
    Beta=len(G[0])
    
    #print('r2:',r)
    if max(r)>0:
        II=[x for x in range(len(G)) if x != ii and P[ii] <= max(r)]
    while max(r)>0:
        if II:
            #print(f"rearranging sections, mutated section:{ii}; reshuffle {II}")
            L={s: Length(G[s]) for s in II}
            min_L=min(L)
            idx= min(L, key=L.get)   #index to the min lengh of the sections 
            #print(f'L:{L}, min_index: {idx}')

            G[idx][jj]= G[idx][jj] + min(math.ceil(r[jj]/P[idx]), math.floor((l_max-Length(G[idx]))/(Y_b[jj]/U)))
            

            II.remove(idx)  #removing the updated section from II
        
        else:
            #print('create new section')
            g= list(np.zeros(Beta,dtype=int))
            p= min(P_max,max(P_min,r[jj]))
            if p> r[jj]:
                g[jj]=1
            else:
                g[jj]=min(math.floor(r[jj]/p), math.floor(l_max/(Y_b[jj]/U)))  #need to update in pseudocode
                #print(g)
            G.append(g)
            P.append(p)
            
        
        r = Update_Res(Q_b,G,P)  #Updating Residual within the while loop

    #print(G,P)
    return G,P


def Mutation(Child, mutation_rate=0.1):

    temp_Ch=deepcopy(Child)
    M_G =temp_Ch['G']
    M_P =temp_Ch['P']
    Beta=len(M_G[0])
    

    for i in range(len(M_P)):
        for j in range(Beta):
            if rng.random()<mutation_rate:
                M_G,M_P = mutate(i,j,M_G,M_P)   #user defined fuction


    return {'G':M_G,'P':M_P}

# C2=Mutation(C)
# print(C2)
# print(C)

## Objective Function

In [9]:
def ObjectiveFunction (chromosome):
    
    G_a_b = chromosome['G']
    P_a = chromosome['P']
    Alpha = len(P_a)             # number of Sections
    

    '''                         Fabric Cost                      '''
        
    # Total fabric length = L # Total Fabric Cost = C_F

    l_a=[Length(G_a_b[alpha]) for alpha in range(Alpha) ] #Length function
    L= np.dot(l_a,P_a)       #Multiply then Sum
    C_F = L*f
    #print('Total Fabric Cost = C_F: ',C_F,'Length:',L)
    
    
    '''                        setup Cost                        '''
    
    
    C_S = 0
    for α in range(Alpha):
        if l_a[α]>e:   # this makes sure that section has at least one garments 
            C_S += c_s       
    #print('Total setup Cost = C_S: ',C_S)

    '''                        excess production Cost                        '''
        
    
    C_E=0
    R=Update_Res(R=Q_b, GG=G_a_b, PP=P_a)
    for r in R:
        if r<0:
            C_E += (-r* c_e)
    #print('Total excess prod Cost = C_E: ',C_E) 
    
    return C_F+C_S+C_E

In [10]:
# ObjectiveFunction(Sol_1)

## Fitness Score

In [11]:
def Fitness(chromosome): 
  
    G_a_b= chromosome['G']
    P_a = chromosome['P']
    Beta= len(G_a_b[0])
    
    score= ObjectiveFunction(chromosome)
    #print('score:',score)
    fitness_score=score
    
                
    '''       Penalty for shortage production           '''
    R= Update_Res(R=demand,GG=G_a_b,PP=P_a)
    for beta in range(Beta):
        if R[beta]>0:
            s_penalty= R[beta]/sum(demand)
            fitness_score +=score*s_penalty/2 
    
    
#     '''         Penalty for excess production           '''
#     r=np.dot(1.02,demand)         # additional 2% allowance
#     R= Update_Res(R=r,GG=G_a_b,PP=P_a)
#     #print(R)
#     for beta in range(Beta):
#         if R[beta]<0:
#             e_penalty= (-R[beta]/sum(demand))   # 2times than s_penalty
#             fitness_score +=score*e_penalty   
            

    
    '''       double check if the solution is valid       '''
    res= Update_Res(R=Q_b,GG=G_a_b,PP=P_a)
    if max(res)>0:
        print('solution is unvalid in demand constraint')
        fitness_score +=10000   #this will eventualy make the solution extinct.

    for i in G_a_b:
        if Length(i)>l_max:
            print('breaking !! fitness !! solution is not valid in length constraint: ',i, Y_b, l_max,Length(i))
            fitness_score +=10000
    
    return fitness_score

# Fitness(Sol_1)    

## Function Initial Population Generation

In [12]:
def GeneratePopulation(pop_size):
    sol1=H3_MB2016(Q=Q_b,Y=Y_b,Pm=[P_min,P_max],U=U,e=e,l_max=l_max)
    sol2=H1_T2020(Q=Q_b,Y=Y_b,Pm=[P_min,P_max],U=U,e=e,l_max=l_max)
    P_of_S=[]
    for sol in (sol1,sol2):
        i=0
        for g in sol['G']:
            if Length(g)>l_max:
                i+=1
        if i==0:
            P_of_S.append(sol)
    
    while len(P_of_S)<pop_size:
        h=rng.integers(0,3)
        #print('h:',h)
        if h==0:
            sol= H1(Q=Q_b,Y=Y_b,Pm=[P_min,P_max],U=U,e=e,l_max=l_max)

        elif h==1:
            sol=H3(Q=Q_b,Y=Y_b,Pm=[P_min,P_max],U=U,e=e,l_max=l_max)

        else:
            sol=H5(Q=Q_b,Y=Y_b,Pm=[P_min,P_max],U=U,e=e,l_max=l_max)
        i=0
        for g in sol['G']:
            if Length(g)>l_max:
                i+=1
        if i==0:
            P_of_S.append(sol)
    return P_of_S
Pool_of_Sol= GeneratePopulation(50)
len(Pool_of_Sol)

50

In [13]:
def S_with_F(p_o_s):
    for i in range(len(p_o_s)): 
        if 'F' not in p_o_s[i]:
            p_o_s[i]['F']=Fitness(p_o_s[i])
    return p_o_s

In [14]:
# P_of_S_with_F= S_with_F(Pool_of_Sol)
# print(type(P_of_S_with_F))

## Tournament Selection

In [15]:
def TournamentSelection(sol_list, k=3):
    '''
    k= number of random parents that take part in one tournament
    k= 3 by default
    '''

    indexes= rng.integers(0,len(sol_list), size=k) 
    
    psbl_parents = [sol_list[i] for i in indexes]
    
    psbl_parents = sorted(psbl_parents, key=lambda d: d['F'])
    return deepcopy(psbl_parents[0])



## GA with Stopping Criteria

In [17]:
def GA_with_TS_with_sc(pop_size, generation,k=3,m_rate=0.05,c_rate=0.95, sc=100):
    
    pool_of_sol= GeneratePopulation(pop_size)
    pool_of_sol= S_with_F(pool_of_sol)

    bests=[]
    it=0
    for g in range(generation):
        for p in range(pop_size):            
            Parent1,Parent2= [TournamentSelection(pool_of_sol,k=k)for i in range(2)]
                    
            child = Crossover(Parent1, Parent2, crossover_rate=c_rate)

            mutated_child=Mutation(child,mutation_rate=m_rate)
            mutated_child['F']=Fitness(mutated_child)
            pool_of_sol.append(mutated_child)
        
        pool_of_sol= sorted(pool_of_sol, key=lambda d: d['F'])
        pool_of_sol = pool_of_sol[:pop_size]
        
        if bests:
            if bests[-1]['F']> pool_of_sol[0]['F']:
                it=0
            else: 
                it+=1
        bests.append(pool_of_sol[0])
        if it> sc:
            print('break')
            break
        

#     x=[i for i in range(len(bests))]
#     yy=[bests[i]['F'] for i in range(len(bests))]
#     fig=px.line(x=x,
#                 y=yy,
#                 title=f'pop_size={pop_size},generation= {generation},k={k},mu_rate= {m_rate},c_rate={c_rate}, best={yy[-1]}, sc={sc}',
#                 labels=dict(x='generation',y='fitness'))
#     fig.show()
    
    return bests[-1]

## Aplying to Tsao et al. dataset

In [18]:
path='/Users/sharif-al-mahmud/Library/Mobile Documents/com~apple~CloudDocs/Thesis/Data/DataSets.xlsx'

In [19]:
df=pd.read_excel(path,sheet_name='Sheet4',index_col=[0])

In [20]:
df=df.T

df.columns = df.columns.str.strip()

col=list(df.columns)
col= sorted(set(col), key=col.index)


In [21]:
df

Style,S4,S4,S4,S4,S5,S5,S5,S5,S5,S7,...,B5,B5,B5,B7,B7,B7,B7,B7,B7,B7
Demand,80.00,95.000,58.000,28.0,25.00,95.000,145.000,130.0,25.000,8.00,...,12498.0,2256.000,569.000,6780.00,14526.000,16473.000,18767.0,112498.000,8569.000,6532.0
Consumption,1.42,1.441,1.462,1.5,1.42,1.441,1.462,1.5,1.523,1.42,...,1.5,1.523,1.634,1.42,1.441,1.462,1.5,1.523,1.634,1.7
Price/unit length,10.00,NaN,NaN,NaN,10.00,NaN,NaN,NaN,NaN,10.00,...,NaN,NaN,NaN,10.00,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
col

['S4', 'S5', 'S7', 'M4', 'M5', 'M7', 'B4', 'B5', 'B7']

## Selected Parameters : 
* k= 2, 
* c_rate= 0.95, 
* m_rate= 0.003, 
* sc= 99 (stoping criteria)
* pop_size= 200, 
* generation= 200

In [24]:
df_result_GA = pd.DataFrame(columns=['style','demand','sol','Fitness','Cost','Runtime'])
k, c_rate, m_rate,sc = (2,0.95,0.003,99) #parameters
pop_size,generation=(200,200)
for j in range(10):
    for i in col:
        demand=df[i].loc['Demand'].to_list()
        demand=list(map(int,demand))
        #print('demand=',demand)

        d=0  #shortange allowance in Percentage
        Q_b= Shortage_allowance(demand,d)

        consumption=df[i].loc['Consumption'].to_list()
        Y_b=list(map(float,consumption))

        if len(Q_b)!=len(Y_b):
            raise ValueError('number of sizes and number of fabric consumption does not match')

        price=df[i].loc['Price/unit length'].to_list()
        price=list(map(float,price))
        f=price[0]

        print(i,demand,Q_b,Y_b,f)
        time_start = time.perf_counter()
        Gbest = GA_with_TS_with_sc(pop_size=pop_size, generation=generation,k=k,m_rate=m_rate,c_rate=c_rate, sc=sc)
        time_elapsed = (time.perf_counter() - time_start)
        print(Gbest, 'time:', time_elapsed)
        #P_cost,P_time= Process(Gbest)
        df_result_GA=df_result_GA.append({'style':i,'demand':demand,'Cost': ObjectiveFunction(Gbest),'sol':Gbest,'Fitness':Gbest['F'],'Runtime': time_elapsed},
                                         ignore_index=True)
    #df_result_GA.to_excel('/Users/sharif-al-mahmud/Library/Mobile Documents/com~apple~CloudDocs/Thesis/Data/GA output for Tsao et al data.xlsx')

S4 [80, 95, 58, 28] [80, 95, 58, 28] [1.42, 1.441, 1.462, 1.5] 10.0
break
{'G': [[2, 3, 0, 1], [6, 0, 1, 0], [0, 1, 5, 0]], 'P': [28, 4, 11], 'F': 5307.530000000001} time: 6.006182625000008
S5 [25, 95, 145, 130, 25] [25, 95, 145, 130, 25] [1.42, 1.441, 1.462, 1.5, 1.523] 10.0
break
{'G': [[0, 3, 4, 5, 1], [0, 4, 9, 0, 0], [5, 0, 0, 1, 0]], 'P': [25, 5, 5], 'F': 7674.599999999999} time: 5.572360375000017
S7 [8, 105, 92, 56, 34, 10, 5] [8, 105, 92, 56, 34, 10, 5] [1.42, 1.441, 1.462, 1.5, 1.523, 1.634, 1.7] 10.0
break
{'G': [[0, 1, 5, 0, 3, 0, 0], [0, 0, 0, 0, 0, 0, 0], [1, 12, 0, 0, 0, 0, 0], [0, 0, 0, 8, 1, 0, 0], [0, 0, 10, 0, 0, 2, 1]], 'P': [9, 5, 8, 7, 5], 'F': 6681.77} time: 13.207427042000006
M4 [654, 443, 938, 245] [654, 443, 938, 245] [1.42, 1.441, 1.462, 1.5] 10.0
break
{'G': [[0, 0, 10, 3], [5, 1, 1, 6], [1, 12, 0, 0], [0, 9, 0, 0], [0, 1, 0, 1], [0, 0, 0, 0], [0, 0, 13, 0], [14, 0, 0, 0]], 'P': [40, 18, 4, 40, 17, 17, 40, 40], 'F': 36558.98999999999} time: 17.442518667
M5 [1

KeyboardInterrupt: 

In [ ]:
df_result_GA